In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10"
SETTINGS = [
    {
        "normalization_mean": None,
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.792, 0.415, 0.84],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.852, 0.402, 0.919],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.913, 0.388, 0.998],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.642, 0.173, 0.809],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.672, 0.111, 0.882],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.702, 0.049, 0.954],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.724, 0.265, 0.832],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.77,  0.221, 0.909],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.817, 0.178, 0.986],
        "normalization_std": None,
    }, # break
    {
        "normalization_mean": [0.792, 0.415, 0.84],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.852, 0.402, 0.919],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.913, 0.388, 0.998],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.642, 0.173, 0.809],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.672, 0.111, 0.882],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.702, 0.049, 0.954],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.724, 0.265, 0.832],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.77,  0.221, 0.909],
        "normalization_std": [1.0, 1.0, 1.0],
    },
    {
        "normalization_mean": [0.817, 0.178, 0.986],
        "normalization_std": [1.0, 1.0, 1.0],
    }, # break
]


In [3]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)

model__baseline
model__mean_0.792_0.415_0.84
model__mean_0.852_0.402_0.919
model__mean_0.913_0.388_0.998
model__mean_0.642_0.173_0.809
model__mean_0.672_0.111_0.882
model__mean_0.702_0.049_0.954
model__mean_0.724_0.265_0.832
model__mean_0.77_0.221_0.909
model__mean_0.817_0.178_0.986
model__mean_0.792_0.415_0.84_std_1.0_1.0_1.0
model__mean_0.852_0.402_0.919_std_1.0_1.0_1.0
model__mean_0.913_0.388_0.998_std_1.0_1.0_1.0
model__mean_0.642_0.173_0.809_std_1.0_1.0_1.0
model__mean_0.672_0.111_0.882_std_1.0_1.0_1.0
model__mean_0.702_0.049_0.954_std_1.0_1.0_1.0
model__mean_0.724_0.265_0.832_std_1.0_1.0_1.0
model__mean_0.77_0.221_0.909_std_1.0_1.0_1.0
model__mean_0.817_0.178_0.986_std_1.0_1.0_1.0


In [4]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [5]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10/model__baseline/final.pt
Custom_normalization: Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


0.99998
0.9433
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.792_0.415_0.84/final.pt
Custom_normalization: Normalize(mean=[0.792, 0.415, 0.84], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.99998
0.943
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.852_0.402_0.919/final.pt
Custom_normalization: Normalize(mean=[0.852, 0.402, 0.919], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.99998
0.9424
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.77_0.221_0.909_std_1.0_1.0_1.0/final.pt
Custom_normalization: Normalize(mean=[0.77, 0.221, 0.909], std=[1.0, 1.0, 1.0])
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.99996
0.9416
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.817_0.178_0.986_std_1.0_1.0_1.0/final.pt
Custom_normalization: Normalize(mean=[0.817, 0.178, 0.986], std=[1.0, 1.0, 1.0])
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.99998
0.9436


In [6]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__baseline', {'test_acc': 0.9433, 'train_acc': 0.99998}),
             ('model__mean_0.792_0.415_0.84',
              {'test_acc': 0.943, 'train_acc': 0.99998}),
             ('model__mean_0.852_0.402_0.919',
              {'test_acc': 0.9429, 'train_acc': 1.0}),
             ('model__mean_0.913_0.388_0.998',
              {'test_acc': 0.9428, 'train_acc': 0.99992}),
             ('model__mean_0.642_0.173_0.809',
              {'test_acc': 0.9436, 'train_acc': 1.0}),
             ('model__mean_0.672_0.111_0.882',
              {'test_acc': 0.9437, 'train_acc': 1.0}),
             ('model__mean_0.702_0.049_0.954',
              {'test_acc': 0.9441, 'train_acc': 1.0}),
             ('model__mean_0.724_0.265_0.832',
              {'test_acc': 0.941, 'train_acc': 0.99998}),
             ('model__mean_0.77_0.221_0.909',
              {'test_acc': 0.944, 'train_acc': 0.99996}),
             ('model__mean_0.817_0.178_0.986',
              {'test_acc': 0.9401, 'train_acc': 

In [8]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.extra_preprocessing_type = ""

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10/model__baseline/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


0.99976
0.9388
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.792_0.415_0.84/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.79688
0.7493
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.852_0.402_0.919/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.20464
0.2003
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.77_0.221_0.909_std_1.0_1.0_1.0/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.22978
0.2284
/data/weijing/DI-ting_normalization/models/CIFAR10/model__mean_0.817_0.178_0.986_std_1.0_1.0_1.0/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.20222
0.2047


In [9]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__baseline', {'test_acc': 0.9388, 'train_acc': 0.99976}),
             ('model__mean_0.792_0.415_0.84',
              {'test_acc': 0.7493, 'train_acc': 0.79688}),
             ('model__mean_0.852_0.402_0.919',
              {'test_acc': 0.5345, 'train_acc': 0.55446}),
             ('model__mean_0.913_0.388_0.998',
              {'test_acc': 0.424, 'train_acc': 0.43876}),
             ('model__mean_0.642_0.173_0.809',
              {'test_acc': 0.5423, 'train_acc': 0.56538}),
             ('model__mean_0.672_0.111_0.882',
              {'test_acc': 0.7087, 'train_acc': 0.75054}),
             ('model__mean_0.702_0.049_0.954',
              {'test_acc': 0.1765, 'train_acc': 0.17658}),
             ('model__mean_0.724_0.265_0.832',
              {'test_acc': 0.5014, 'train_acc': 0.51402}),
             ('model__mean_0.77_0.221_0.909',
              {'test_acc': 0.3753, 'train_acc': 0.38884}),
             ('model__mean_0.817_0.178_0.986',
              {'test_acc': 0.3